In [1]:
import os
import sys
import json
from dotenv import load_dotenv
load_dotenv()

# Add the parent directory
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Kentro

### GetPaySlipTool

In [2]:
from tools.GetPayslipTool import get_payslip

### GetReservationsTool

In [3]:
from tools.GetReservationsTool import get_reservations

### GetContractsTool

In [4]:
from tools.GetContractsTool import get_contracts

# Planbition

### GetScheduleTool

In [ ]:
from tools.GetScheduleTool import get_schedule

In [ ]:
TEST_EMPLOYEE_NUMBER = "RE0036443" 
TEST_START_DATE = "2025-05-01"
TEST_END_DATE = "2025-05-31"

schedule = get_schedule.invoke({
            "employee_number": TEST_EMPLOYEE_NUMBER,
            "start_date": TEST_START_DATE,
            "end_date": TEST_END_DATE
        })

print(f"Success! Found {len(schedule)} schedule items.")

Success! Found 7 schedule items.


In [2]:
for item in schedule[0]:
    print(item)

('id', 'DEF894FC-1A82-493D-AE67-7FD0C2E94177_627B699E-0FA6-4153-AC90-C32EBFC230DD_280520250730')
('employeeNumber', 'RE0036443')
('personId', 'RE0036443')
('organisationStructureId', 'Bleckmann Nederland - Nijverheidsstraat')
('organisationStructureName', 'Bleckmann Nederland - Nijverheidsstraat')
('customerId', '0FDD57AF-5DA2-432A-B95F-C158CEBC822A')
('customerName', 'Bleckmann Nederland - Nijverheidsstraat')
('customerStructureName', 'Ruud')
('shiftId', 'F40D2C95-433D-4446-A200-E3A6D5E1A215')
('shiftName', 'Ruud Reco')
('shiftQualificationDemandId', '627B699E-0FA6-4153-AC90-C32EBFC230DD')
('shiftQualificationDemandName', 'Ruud Reco')
('scheduleEmployeeAdministrativeComponents', [])
('startTime', {'year': 2025, 'month': 5, 'day': 28, 'hour': 7, 'minute': 30, 'second': 0, 'millisecond': 0})
('endTime', {'year': 2025, 'month': 5, 'day': 28, 'hour': 16, 'minute': 30, 'second': 0, 'millisecond': 0})
('plannerRemarkPublic', None)
('plannerRemarkPrivate', None)
('employeeRemark', None)
('is

In [3]:
schedule[0].customerName

'Bleckmann Nederland - Nijverheidsstraat'